# Prerequisites

- Python 3.10.4

> Warning: Installation from conda environment may take few minutes

Configuring conda environment

```cmd
conda activate ca2_env
conda install matplotlib
conda install pandas
conda install nltk
conda install scikit-learn
conda install jsonpickle
conda install -c conda-forge textblob
conda install -c conda-forge tweepy
conda install -c conda-forge pycountry
conda install -c conda-forge wordcloud
conda install -c conda-forge langdetect
```

References:
https://towardsdatascience.com/step-by-step-twitter-sentiment-analysis-in-python-d6f650ade58d




In [3]:
# Import Libraries
from textblob import TextBlob
import sys
import tweepy
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import jsonpickle
import os
import nltk
import pycountry
import re
import string

from wordcloud import WordCloud, STOPWORDS
from PIL import Image
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from langdetect import detect
from nltk.stem import SnowballStemmer
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from sklearn.feature_extraction.text import CountVectorizer

In [2]:
#nltk.download('brown')
#nltk.download('vader_lexicon')

In [39]:
# Authentication
api_key = "Y0hENmxMwkILuxxY0yb9NHG0L"
api_secret = "33OSFPPFNWlN3S9CvL5ok4ClBLjUdviC4zzpLs9aRBwJVzHPMA"
bearerToken = "AAAAAAAAAAAAAAAAAAAAABoHcAEAAAAA78rV2Phg2vUbwTdcJ2QPGGji65M%3DsTr8DETODttIuwkHy9czi663YUpUPLDOVY2VqUzmBdshiJmLT3"

auth = tweepy.OAuth2BearerHandler(bearerToken)
api = tweepy.API(auth)

In [56]:
#Download tweets to json file
def percentage(part,whole):
     return 100 * float(part)/float(whole)

query = "(agriculture OR farmers) (price OR cost) (place_country:IE) (lang:en)"
environment = "development"
fromDate = "202201010000"
tweets = tweepy.Cursor(api.search_full_archive,label=environment,query=query,fromDate=fromDate).items()

In [92]:
# Testing 7 days search
#tweets = api.search_tweets(q="(agriculture OR farmers) (price OR cost) place_country:IE lang:en -is:retweet", lang="en", result_type="mixed", count=100)
tweets = tweepy.Cursor(api.search_tweets,q="(agriculture OR farmers) (price OR cost) lang:en -is:retweet", lang="en", result_type="mixed", count=100).items(1)

In [93]:
# Save tweets
pd.json_normalize([t._json for t in tweets]).to_csv("../data/twitter/agriculture_farmers_2022.csv")

In [94]:
# Read from csv
tweets_pd = pd.read_csv("../data/twitter/agriculture_farmers_2022.csv", index_col=0)

In [95]:
tweets_pd.columns

Index(['created_at', 'id', 'id_str', 'text', 'truncated', 'source',
       'in_reply_to_status_id', 'in_reply_to_status_id_str',
       'in_reply_to_user_id', 'in_reply_to_user_id_str',
       'in_reply_to_screen_name', 'geo', 'coordinates', 'place',
       'contributors', 'is_quote_status', 'retweet_count', 'favorite_count',
       'favorited', 'retweeted', 'lang', 'entities.hashtags',
       'entities.symbols', 'entities.user_mentions', 'entities.urls',
       'metadata.result_type', 'metadata.iso_language_code', 'user.id',
       'user.id_str', 'user.name', 'user.screen_name', 'user.location',
       'user.description', 'user.url', 'user.entities.description.urls',
       'user.protected', 'user.followers_count', 'user.friends_count',
       'user.listed_count', 'user.created_at', 'user.favourites_count',
       'user.utc_offset', 'user.time_zone', 'user.geo_enabled',
       'user.verified', 'user.statuses_count', 'user.lang',
       'user.contributors_enabled', 'user.is_translator'

In [96]:
tweets_pd.columns[tweets_pd.columns.str.endswith("text")]

Index(['text'], dtype='object')

In [97]:
for c in tweets_pd.columns:
    print(c)

created_at
id
id_str
text
truncated
source
in_reply_to_status_id
in_reply_to_status_id_str
in_reply_to_user_id
in_reply_to_user_id_str
in_reply_to_screen_name
geo
coordinates
place
contributors
is_quote_status
retweet_count
favorite_count
favorited
retweeted
lang
entities.hashtags
entities.symbols
entities.user_mentions
entities.urls
metadata.result_type
metadata.iso_language_code
user.id
user.id_str
user.name
user.screen_name
user.location
user.description
user.url
user.entities.description.urls
user.protected
user.followers_count
user.friends_count
user.listed_count
user.created_at
user.favourites_count
user.utc_offset
user.time_zone
user.geo_enabled
user.verified
user.statuses_count
user.lang
user.contributors_enabled
user.is_translator
user.is_translation_enabled
user.profile_background_color
user.profile_background_image_url
user.profile_background_image_url_https
user.profile_background_tile
user.profile_image_url
user.profile_image_url_https
user.profile_banner_url
user.profile_

In [98]:
tweets_pd.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1 entries, 0 to 0
Data columns (total 70 columns):
 #   Column                                   Non-Null Count  Dtype  
---  ------                                   --------------  -----  
 0   created_at                               1 non-null      object 
 1   id                                       1 non-null      int64  
 2   id_str                                   1 non-null      int64  
 3   text                                     1 non-null      object 
 4   truncated                                1 non-null      bool   
 5   source                                   1 non-null      object 
 6   in_reply_to_status_id                    0 non-null      float64
 7   in_reply_to_status_id_str                0 non-null      float64
 8   in_reply_to_user_id                      0 non-null      float64
 9   in_reply_to_user_id_str                  0 non-null      float64
 10  in_reply_to_screen_name                  0 non-null   

In [100]:
# Replace truncated with extended text
tweets_pd["text"] = tweets_pd.apply(lambda r: r["extended_tweet.full_text"] if r["truncated"] else r["text"], axis=1)
if("quoted_status.text" in tweets_pd.columns):
    tweets_pd["quoted_status.text"] = tweets_pd.apply(lambda r: r["quoted_status.extended_tweet.full_text"] if r["quoted_status.truncated"] else r["quoted_status.text"], axis=1)


> split dataset by columns. Get quoted status info

In [101]:
# Status tweets
tweets_status_pd = tweets_pd[[c for c in tweets_pd.columns if not(c.startswith("quoted"))]]
tweets_status_pd.rename({"place.full_name":"place"}, axis=1, inplace=True)


In [102]:
if("quoted_status.text" in tweets_pd.columns):
    # Quote status tweets
    tweets_quoted_status_pd = tweets_pd[tweets_pd["is_quote_status"]][tweets_pd.columns[tweets_pd.columns.str.startswith("quoted")]]
    # rename columns
    tweets_quoted_status_pd.rename(mapper=lambda c: c.replace("quoted_status.",""), axis='columns', inplace=True)
    # add dummy "place.bounding_box.type" and "place.bounding_box.coordinates"
    tweets_quoted_status_pd["place.bounding_box.type"] = None
    tweets_quoted_status_pd["place.bounding_box.coordinates"] = None

In [103]:
if("quoted_status.text" in tweets_pd.columns):
    columns = ["id","geo","coordinates","reply_count","favorite_count", "place", "text" ,"user.location", "place.bounding_box.type", "place.bounding_box.coordinates"]
    arr1 = tweets_quoted_status_pd[columns].to_numpy()
    arr2 = tweets_status_pd[columns].to_numpy()
    tweets_final_pd = pd.DataFrame(np.concatenate((arr1,arr2), axis=0), columns = columns)
else:
    tweets_final_pd =  tweets_status_pd

In [104]:
tweets_final_pd.to_csv("final.csv")

In [109]:
tweets_final_pd

,created_at,id,id_str,text,truncated,source,in_reply_to_status_id,in_reply_to_status_id_str,in_reply_to_user_id,in_reply_to_user_id_str,...,user.profile_text_color,user.profile_use_background_image,user.has_extended_profile,user.default_profile,user.default_profile_image,user.following,user.follow_request_sent,user.notifications,user.translator_type,user.withheld_in_countries
0,Sun May 01 10:24:24 +0000 2022,1520710722208083968,1520710722208083968,russians attempt to buy grain from Zaporizhzhi...,False,"<a href=""http://twitter.com/download/android"" ...",NaN,NaN,NaN,NaN,...,0,False,False,False,False,NaN,NaN,NaN,none,[]


In [114]:
type(tweets_final_pd)

pandas.core.frame.DataFrame

In [134]:
noOfTweet = tweets_final_pd.shape[0]
#Sentiment Analysis
positive = 0
negative = 0
neutral = 0
polarity = 0
tweet_list = []
neutral_list = []
negative_list = []
positive_list = []
for index, tweet in tweets_final_pd.iterrows():
    tweet_list.append(tweet["text"])
    analysis = TextBlob(tweet["text"])
    score = SentimentIntensityAnalyzer().polarity_scores(tweet["text"])
    neg = score['neg']
    neu = score['neu']
    pos = score['pos']
    comp = score['compound']
    polarity += analysis.sentiment.polarity
 
    if neg > pos:
        negative_list.append(tweet["text"])
        negative += 1
    elif pos > neg:
        positive_list.append(tweet["text"])
        positive += 1
    elif pos == neg:
        neutral_list.append(tweet.text)
        neutral += 1

positive = percentage(positive, noOfTweet) if positive > 0 else 0
negative = percentage(negative, noOfTweet) if negative > 0 else 0
neutral = percentage(neutral, noOfTweet) if neutral > 0 else 0
polarity = percentage(polarity, noOfTweet) if polarity > 0 else 0
positive = format(positive, '.1f')
negative = format(negative, '.1f')
neutral = format(neutral, '.1f')

In [135]:
#Number of Tweets (Total, Positive, Negative, Neutral)
tweet_list = pd.DataFrame(tweet_list)
neutral_list = pd.DataFrame(neutral_list)
negative_list = pd.DataFrame(negative_list)
positive_list = pd.DataFrame(positive_list)
print("total number: ",len(tweet_list))
print("positive number: ",len(positive_list))
print("negative number: ", len(negative_list))
print("neutral number: ",len(neutral_list))

total number:  1
positive number:  0
negative number:  1
neutral number:  0
